In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

import var_optimise

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

In [2]:
lih = HamiltonianSmall('LiH', 1.5)
beh2 = HamiltonianSmall('BeH2', 1.3)

h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

water_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')
water_parity = Hamiltonian('H2O_STO3g_14qubits', 'parity')
water_bk = Hamiltonian('H2O_STO3g_14qubits', 'bk')

ammonia_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')
ammonia_parity = Hamiltonian('NH3_STO3g_16qubits', 'parity')
ammonia_bk = Hamiltonian('NH3_STO3g_16qubits', 'bk')

In [3]:
pr = lih.pauli_rep
n = pr.num_qubits
energy, state = pr.ground()
β = var_optimise.optimal_beta(pr)
objective = var_optimise.objective(pr, β) # the cost function's value
print("beta:", β)
print("objective:", objective)

beta: {0: [0.2627877783884178, 0.25172728999060096, 0.4854849316209812], 1: [0.2731407677187616, 0.2708496665000182, 0.45600956578122026], 2: [0.2627877734835786, 0.25172728735894984, 0.4854849391574715], 3: [0.2731407699348335, 0.2708496658369026, 0.45600956422826394]}
objective: 2.6472360929681003


In [4]:
def prod_inverse(Q, β):
    # prod_{j in supp(Q)} beta_{j, Q_j}^{-1}
    pauli_to_index = {'X': 0, 'Y': 1, 'Z': 2}
    prod = 1.0
    for i in range(len(Q)):
        if Q[i] != 'I':
            index = pauli_to_index[Q[i]]
            b = β[(len(Q)-1)-i][index]  # qiskit ordering: if i=0 then need to look up beta[n]
            prod *= b
    return prod**(-1)

def lagrange_restriction_numerator(i, p, dic, β):
    tally = 0.0
    for Q in dic:
        if Q[(len(Q)-1)-i] == p: # qiskit ordering
            tally += (dic[Q]**2) * prod_inverse(Q, β)
    return tally

def lagrange_restriction_denominator(i, dic, β):
    tally = 0.0
    for Q in dic:
        if Q[(len(Q)-1)-i] != "I": # qiskit ordering
            tally += (dic[Q]**2) * prod_inverse(Q, β)
    return tally

def lagrange_restriction(i, p, dic, β, objective):
    numerator = lagrange_restriction_numerator(i, p, dic, β)
    denominator = lagrange_restriction_denominator(i, dic, β)
    return numerator / denominator

In [5]:
lagrange_restriction(0, 'X', pr.dic, β, objective)

0.2627877817553151

In [6]:
lagrange_restriction(0, 'Y', pr.dic, β, objective)

0.2517272976434559

In [7]:
lagrange_restriction(0, 'Z', pr.dic, β, objective)

0.485484920601229